In [ ]:
import numpy as np
import pandas as pd
import datetime
#import missingno as msno
from pandas import DataFrame
from pandas import concat

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Extra Libs
import seaborn as sns
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import plotly.express as px
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.models import HoverTool
from IPython.display import HTML, display

# Default visual settings

plt.rcParams["font.family"] = "serif"
plt.rcParams['figure.dpi'] = 150

# Ignore warnings

import warnings
warnings.filterwarnings("ignore")



In [ ]:

# reading xlsx file
df_RH=pd.read_excel("C:/Users/abdbe/Documents/Article ait melloul/RH101W.xlsx")
df_ZI=pd.read_excel("C:/Users/abdbe/Documents/Article ait melloul/ZI101W.xlsx")

In [ ]:
df_ZI

In [ ]:
df_RH

In [ ]:
df_RH = df_RH.set_index(pd.DatetimeIndex(df_RH['Time']))

df_ZI = df_ZI.set_index(pd.DatetimeIndex(df_ZI['Time']))

In [ ]:
df_RH['Year'] = df_RH['Time'].apply(lambda time: time.year)
df_RH['Month'] = df_RH['Time'].apply(lambda time: time.month)
df_RH['Day'] = df_RH['Time'].apply(lambda t: t.day)
df_RH['Hour'] = df_RH['Time'].apply(lambda time: time.hour)
df_RH['Day of Week'] = df_RH['Time'].apply(lambda time: time.dayofweek)

df_ZI['Day of Week'] = df_ZI['Time'].apply(lambda time: time.dayofweek)
df_ZI['Year'] = df_ZI['Time'].apply(lambda time: time.year)
df_ZI['Month'] = df_ZI['Time'].apply(lambda time: time.month)
df_ZI['Day'] = df_ZI['Time'].apply(lambda t: t.day)
df_ZI['Hour'] = df_ZI['Time'].apply(lambda time: time.hour)

In [ ]:
df_RH = df_RH.drop(['Time'], axis=1)

df_ZI = df_ZI.drop(['Time'], axis=1)

In [ ]:
df_RH=df_RH.resample('H').mean()
df_ZI=df_ZI.resample('H').mean()

In [ ]:
df_ZI.describe()

In [ ]:
def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_tail = q1 - 1.5 * iqr
    upper_tail = q3 + 1.5 * iqr
    return df[column].apply(lambda x: np.nan if x < lower_tail or x > upper_tail else x)

columns_to_check = ['PM2.5', 'PM10', 'PM1', 'CO', 'CO2']

# Copy original dataframes
df_cleaned_Out_RH = df_RH.copy()
df_cleaned_Out_ZI = df_ZI.copy()

# Apply the function to each column in df_cleaned_Out_RH
for col in columns_to_check:
    df_cleaned_Out_RH[col] = remove_outliers(df_cleaned_Out_RH, col)

# Apply the function to each column in df_cleaned_Out_ZI
for col in columns_to_check:
    df_cleaned_Out_ZI[col] = remove_outliers(df_cleaned_Out_ZI, col)

print(df_cleaned_Out_RH)
print(df_cleaned_Out_ZI)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))
def optimize_k(data, target):
    errors = []
    for k in range(1, 40, 1):
        imputer = KNNImputer(n_neighbors=k)
        imputed = imputer.fit_transform(data)
        df_imputed = pd.DataFrame(imputed, columns=data.columns)
        
        X = df_imputed.drop(target, axis=1)
        y = df_imputed[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        error = rmse(y_test, preds)
        errors.append({'K': k, 'RMSE': error})
        
    return errors

In [ ]:
k_errors = optimize_k(data=df_cleaned_Out_ZI, target='PM2.5')

In [ ]:
k_errors

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=4)
imputed = imputer.fit_transform(df_cleaned_Out_ZI)
df_cleaned_ZI = pd.DataFrame(imputed, columns=df_cleaned_Out_ZI.columns)

In [ ]:
imputer = KNNImputer(n_neighbors=8)
imputed = imputer.fit_transform(df_cleaned_Out_RH)
df_cleaned_RH = pd.DataFrame(imputed, columns=df_cleaned_Out_RH.columns)

In [ ]:
dataRH=df_cleaned_RH
dataZI=df_cleaned_ZI

In [ ]:
dataZI

In [ ]:

dataRH["date"]=pd.to_datetime(dataRH[['Year', 'Month', 'Day', 'Hour']])      
dataZI["date"]=pd.to_datetime(dataZI[['Year', 'Month', 'Day', 'Hour']])  

dataRH = dataRH.set_index(pd.DatetimeIndex(dataRH['date']))
dataZI = dataZI.set_index(pd.DatetimeIndex(dataZI['date']))

dataRH = dataRH.drop(['date'], axis=1)
dataZI = dataZI.drop(['date'], axis=1)

In [ ]:
dataZI

In [ ]:
dataRH

# Prediction

## LGBMRegressor 


In [ ]:
X = dataZI.drop(columns = ['PM2.5','PM10'])
y = dataZI['PM2.5']

In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE

random.seed(0)

# Define the search space
search_spaces = {
    'num_leaves': Integer(10, 50),
    'min_child_samples': Integer(1, 100),
    'subsample': Real(1e-8, 1.0, 'log-uniform'),
    'colsample_bytree': Real(1e-8, 1.0, 'log-uniform'),
    'reg_alpha': Real(1e-8, 1.0, 'log-uniform'),
    'reg_lambda': Real(1e-8, 1.0, 'log-uniform'),
    'learning_rate': Real(1e-8, 1.0, 'log-uniform'),
    'max_depth': Integer(-1, 50),
    'n_estimators': Integer(10, 2000)
}

# Initialize BayesSearchCV
lgbm_opt = BayesSearchCV(
    estimator=LGBMRegressor(),
    search_spaces=search_spaces,
    n_iter=100,
    cv=5,
    random_state=0
)

# Fit the model
lgbm_opt.fit(X, y)

# Get the best estimator
best_gbm = lgbm_opt.best_estimator_

# Predict using the best estimator
lgbm_tuned_fcst = best_gbm.predict(X)

# Calculate and print R^2 score
r2 = r2_score(y, lgbm_tuned_fcst)
print(f"R^2 score: {r2}")

# Feature importance from the best estimator
fea_imp_ = pd.DataFrame({'cols': X.columns, 'fea_imp': best_gbm.feature_importances_})
fea_imp_sorted = fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by='fea_imp', ascending=False)

# Plot feature importances
plt.figure(figsize=(20, 10))
sns.barplot(x="fea_imp", y="cols", data=fea_imp_sorted)
plt.title('LightGBM Feature Importances')
plt.tight_layout()
plt.show()

# Save the plot
plt.savefig('lgbm_importances-01.png')

# Create a DataFrame of sorted feature importances for further use if needed
feature_importances_ = pd.DataFrame(sorted(zip(best_gbm.feature_importances_, X.columns)), columns=['Value', 'Feature'])

# Recursive Feature Elimination (RFE)

# Create the RFE model and select 7 attributes
rfe = RFE(estimator=best_gbm, n_features_to_select=7)
rfe = rfe.fit(X, y)

# Summarize the selection of the attributes
print("Selected features:", rfe.support_)

# Summarize the ranking of the attributes
fea_rank_ = pd.DataFrame({'cols': X.columns, 'fea_rank': rfe.ranking_})
fea_rank_sorted = fea_rank_.sort_values(by='fea_rank', ascending=True)

# Print the ranking of features
print(fea_rank_sorted)

# Optional: Print only the selected features
selected_features = fea_rank_sorted[fea_rank_sorted['fea_rank'] == 1]
print("Selected features ranked as 1:")
print(selected_features)


In [ ]:
RFE_data=dataZI[['RH','RT','windgust','PM2.5' ,'winddir','CO2','humidity']]

# mRMR

In [ ]:
X = dataZI.drop(columns = ['PM2.5','PM10'])
y = dataZI['PM2.5']

In [ ]:
X_imputed_df = pd.DataFrame(X, columns = X.columns)

In [ ]:
import mrmr
from mrmr import mrmr_regression

In [ ]:
SF = mrmr_regression(X_imputed_df, y, K = 7)

In [ ]:
SF

In [ ]:
Mr_data=dataZI[['PM2.5', 'windgust', 'temp', 'feelslike', 'sealevelpressure', 'humidity', 'CO2']]


In [ ]:
Mr_data

# Modelation

In [ ]:
def series_to_supervised(data, train_size=0.5, n_in=1, n_out=1, target_column='target', dropnan=True, scale_X=True):

    df = data.copy()

    # Make sure the target column is the last column in the dataframe
    df['PM2'] = df[target_column] # Make a copy of the target column
    df = df.drop(columns=[target_column]) # Drop the original target column

    target_location = df.shape[1] - 1 # column index number of target

    # ...X
    #X = df.iloc[:, :target_location]
    X = df.iloc[:,:]

    # ...y
    y = df.iloc[:, [target_location]]

    # Scale the features
    if scale_X:
        #col_names=['target']
        #features = X[col_names]
        features = X[X.columns]
        scalerX = MinMaxScaler().fit(features.values)
        features = scalerX.transform(features.values)

        #X['target'] = features
        X[X.columns] = features

    #n_vars_x = X.shape[1]
    x_vars_labels = X.columns
    y_vars_labels = y.columns

    x_cols, x_names = list(), list()
    y_cols, y_names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        x_cols.append(X.shift(i))
        x_names += [('%s(t-%d)' % (j, i)) for j in x_vars_labels]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        y_cols.append(y.shift(-i))
        if i == 0:
            y_names += [('%s(t)' % (j)) for j in y_vars_labels]
        else:
            y_names += [('%s(t-%d)' % (j, i)) for j in y_vars_labels]

    # put it all together
    x_agg = pd.concat(x_cols, axis=1)
    x_agg.columns = x_names

    y_agg = pd.concat(y_cols, axis=1)
    y_agg.columns = y_names

    agg=pd.concat([x_agg,y_agg], axis=1)
    agg.columns = x_names + y_names
    #print(agg)


    # drop rows with NaN values
    if dropnan:
        x_agg.dropna(inplace=True)
        y_agg.dropna(inplace=True)

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    """
    diff = y_agg.shape[0] - x_agg.shape[0]
    idx = [i for i in range(0, diff)]
    y_agg = y_agg.drop(df.index[idx])"""

    nf = X.shape[1]
    xx = agg.iloc[:,:n_in*nf]
    yy = agg.iloc[:,-n_out:]

    split_index = int(xx.shape[0]*train_size) # the index at which to split df into train and test

    # ...train
    X_train = xx.iloc[:split_index, :]
    y_train = yy.iloc[:split_index, ]

    # ...test
    X_test = xx.iloc[split_index:, :] # original is split_index:-1
    y_test = yy.iloc[split_index:, ] # original is split_index:-1

    return X_train, y_train, X_test, y_test, scale_X

def smape(y_true, y_pred):
    return 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

def calculate_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_explained_variance(y_true, y_pred):
    return np.var(y_true - y_pred) / np.var(y_true)

def calculate_medae(y_true, y_pred):
    return np.median(np.abs(y_true - y_pred))




In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from lightgbm import LGBMRegressor 
from sklearn.linear_model import ElasticNet
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import mean_absolute_error,mean_squared_error

import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

import sklearn.metrics

from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score, median_absolute_error



In [ ]:
#dataset=dataZI
dataset=RFE_data
#dataset=Mr_data


In [ ]:
X_train, y_train, X_test, y_test, scale_X = series_to_supervised(dataset, train_size=0.7, n_in=24, n_out=1, target_column='PM2.5', dropnan=True, scale_X=True)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tabulate import tabulate



# Fit and evaluate models
models = {
    "LinearRegression": MultiOutputRegressor(LinearRegression()),
    "RandomForestRegressor": MultiOutputRegressor(RandomForestRegressor(max_depth=26, min_samples_leaf=16, min_samples_split=43, n_estimators=400)),
    "GradientBoostingRegressor": MultiOutputRegressor(GradientBoostingRegressor(learning_rate=0.08357797442759243, n_estimators=400, max_depth=26, min_samples_leaf=16, min_samples_split=43)),
    "SVR": MultiOutputRegressor(SVR(C=7.2184767589092855, epsilon=0.0004140476127681186, gamma=0.029462473319069343)),
    "LGBMRegressor": MultiOutputRegressor(LGBMRegressor(reg_lambda=12.203617911779522, alpha=11.974507437260828, colsample_bytree=1, colsample_bynode=0.8, learning_rate=0.02, n_estimators=800, max_depth=4, min_child_weight=72, subsample=0.5382348429040217)),
    "XGBRegressor": MultiOutputRegressor(XGBRegressor(reg_lambda=0.0014159666728948113, alpha=0.035222713638458546, colsample_bytree=0.902004429582736, colsample_bynode=0.7786908529650358, learning_rate=0.017244790780114542, n_estimators=581, max_depth=5, min_child_weight=121, subsample=0.7985209953151778))
}

results = []

for name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)
    
    # Predict on test
    test_pred = model.predict(X_test)
    
    # Collect metrics
    metrics = {
        'Model': name,
        'R2 Test': round(r2_score(y_test, test_pred), 4),
        'RMSE Test': round(np.sqrt(mean_squared_error(y_test, test_pred)), 4),
        'MAE Test': round(mean_absolute_error(y_test, test_pred), 4),
        #'SMAPE Test': round(smape(y_test, test_pred), 4),
        'MSE Test': round(calculate_mse(y_test, test_pred), 4),
        #'MAPE Test': round(calculate_mape(y_test, test_pred), 4),
        'Explained Variance Test': round(calculate_explained_variance(y_test, test_pred), 4),
        'MedAE Test': round(calculate_medae(y_test, test_pred), 4)
    }
    
    results.append(metrics)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame as a formatted table
print(tabulate(results_df, headers='keys', tablefmt='grid'))


In [ ]:
#fit the model on train data 
LGBM=MultiOutputRegressor(LGBMRegressor( reg_lambda= 12.203617911779522, alpha = 11.974507437260828, colsample_bytree= 1,
colsample_bynode= 0.8,learning_rate= 0.02, n_estimators= 800, max_depth= 4, min_child_weight= 72, subsample= 0.5382348429040217))

LGBM.fit(X_train,y_train)
#predict on train 
train_lgbm = LGBM.predict(X_train)
test_lgbm = LGBM.predict(X_test)
print('-'*50)
#fit the model on train data 
XGB=XGBRegressor( reg_lambda= 0.0014159666728948113, alpha = 0.035222713638458546, colsample_bytree= 0.902004429582736,
 colsample_bynode= 0.7786908529650358,learning_rate= 0.017244790780114542, n_estimators= 581, max_depth= 5, min_child_weight= 121, subsample= 0.7985209953151778)
XGB.fit(X_train,y_train)
#predict on train 
train_xgb = XGB.predict(X_train)
test_xgb = XGB.predict(X_test)
print('-'*50)

#fit the model on train data 
SV=SVR(C=7.2184767589092855, epsilon=0.0004140476127681186,
    gamma=0.029462473319069343)
SV.fit(X_train,y_train)
#predict on train 
train_SVR = SV.predict(X_train)
#predict on test
test_SVR = SV.predict(X_test)
#accuracy on test
print('-'*50)

#fit the model on train data 
GB=GradientBoostingRegressor(learning_rate=0.08357797442759243, n_estimators=400,max_depth=26, min_samples_leaf=16, min_samples_split=43)
GB.fit(X_train,y_train)
#predict on train 
train_GB = GB.predict(X_train)
#predict on test
test_GB = GB.predict(X_test)

print('-'*50)

#fit the model on train data 
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

#predict on train 
train_lin = lin_reg.predict(X_train)
#predict on test
test_lin = lin_reg.predict(X_test)
print('-'*50)

#fit the model on train data 
RF=RandomForestRegressor(max_depth=26, min_samples_leaf=16, min_samples_split=43,n_estimators=400)
#RF=MultiOutputRegressor(RandomForestRegressor())
RF.fit(X_train,y_train)
#predict on train 
train_rf = RF.predict(X_train)
#predict on test
test_rf = RF.predict(X_test)

print('-'*50)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Create fig size for plotting
fig, ax = plt.subplots(figsize=(20, 10))

# Set the style
plt.style.use('seaborn-whitegrid')

# Plot the true values 
ax.plot(y_test.index, y_test, color='black', label='Observed', marker='o', markerfacecolor='black', markersize=8, linestyle='-', linewidth=2)

# Plot the predictions
ax.plot(y_test.index, test_lgbm, color='#1f77b4', label='LightGBM', marker='s', markerfacecolor='#1f77b4', markersize=8, linestyle='--', linewidth=2)

# Set title and labels
plt.title('Comparison of Observed and Predicted PM2.5 Concentrations', fontweight="bold", fontsize=24, pad=20)
ax.set_xlabel('Date and Time', fontweight="bold", fontsize=20, labelpad=15)
ax.set_ylabel('PM2.5 Concentration (µg/m³)', fontweight="bold", fontsize=20, labelpad=15)

# Customize the ticks
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

# Set legend
ax.legend(loc="upper right", fontsize=16, title="Legend", title_fontsize='16')

# Format the x-axis with date labels
xfmt = mdates.DateFormatter('%Y-%m-%d %H:%M')
ax.xaxis.set_major_formatter(xfmt)
plt.xticks(rotation=45)

# Add grid lines for better readability
ax.grid(axis='both', linestyle='--', linewidth=0.5)

# Remove top and right spines for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)

# Save the figure with high resolution
plt.savefig("fit_curve_PM25.png", dpi=300, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Create fig size for plotting
fig, ax = plt.subplots(figsize=(20, 10))

# Set the style
plt.style.use('seaborn-whitegrid')

# Plot the true values 
ax.plot(y_test[:24].index, y_test[:24], color='black', label='True', marker='o', markerfacecolor='black', markersize=8, linestyle='-', linewidth=2)

# Plot the predictions
ax.plot(y_test[:24].index, test_lgbm[:24], color='#1f77b4', label='LightGBM', marker='s', markerfacecolor='#1f77b4', markersize=8, linestyle='--', linewidth=2)
ax.plot(y_test[:24].index, test_GB[:24], color='#ff7f0e', label='GBRT', marker='D', markerfacecolor='#ff7f0e', markersize=8, linestyle='--', linewidth=2)
ax.plot(y_test[:24].index, test_xgb[:24], color='#2ca02c', label='XGBoost', marker='^', markerfacecolor='#2ca02c', markersize=8, linestyle='--', linewidth=2)
ax.plot(y_test[:24].index, test_rf[:24], color='#d62728', label='RF', marker='v', markerfacecolor='#d62728', markersize=8, linestyle='--', linewidth=2)
ax.plot(y_test[:24].index, test_SVR[:24], color='#9467bd', label='SVR', marker='p', markerfacecolor='#9467bd', markersize=8, linestyle='--', linewidth=2)

# Set title and labels
plt.title('LightGBM Feature Selection with 24h Lags', fontweight="bold", fontsize=24, pad=20)
ax.set_xlabel('Hour', fontweight="bold", fontsize=20, labelpad=15)
ax.set_ylabel('PM2.5 Concentration (µg/m³)', fontweight="bold", fontsize=20, labelpad=15)

# Customize the ticks
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

# Set legend
ax.legend(loc="upper right", fontsize=16, title="Models", title_fontsize='16')

# Format the x-axis with date labels
xfmt = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(xfmt)

# Add grid lines for better readability
ax.grid(axis='both', linestyle='--', linewidth=0.5)

# Remove top and right spines for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)

# Save the figure with high resolution
plt.savefig("observed_predicted_PM25.png", dpi=300, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()


In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have a fitted MultiOutputRegressor model (LGBM)
# and your X_test is the test set

# Create a TreeExplainer for each output
shap_values_list = []
for estimator in LGBM.estimators_:
    explainer = shap.TreeExplainer(estimator)
    shap_values = explainer.shap_values(X_test)
    shap_values_list.append(shap_values)

# Plot SHAP values for the first output as an example
shap_values = shap_values_list[0]

# Set the style
plt.style.use('seaborn-whitegrid')

# Adjust the height by changing the figsize parameter (width, height)
fig, ax = plt.subplots(figsize=(15, 5))
shap.summary_plot(
    shap_values, X_test, feature_names=X_test.columns, plot_type="bar", show=False
)

# Customize the plot to match PLOS ONE style
plt.title("Feature Importance based on SHAP Values", fontsize=20, weight='bold', pad=20)
plt.xlabel('Mean |SHAP Value| (Impact on Model Output)', fontsize=16, weight='bold', labelpad=15)
plt.ylabel('Features', fontsize=16, weight='bold', labelpad=15)

# Customize ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Remove top and right spines for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)

# Adjust layout
plt.tight_layout()

# Save the figure with high resolution
plt.savefig("feature_importance_plot.png", dpi=300, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()


In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have a fitted MultiOutputRegressor model (LGBM)
# and your X_test is the test set

# Create a TreeExplainer for each output
shap_values_list = []
for estimator in LGBM.estimators_:
    explainer = shap.TreeExplainer(estimator)
    shap_values = explainer.shap_values(X_test)
    shap_values_list.append(shap_values)

# Plot SHAP values for the first output as an example
shap_values = shap_values_list[0]

# Set the style
plt.style.use('seaborn-whitegrid')

# Adjust the height by changing the figsize parameter (width, height)
fig, ax = plt.subplots(figsize=(15, 5))
shap.summary_plot(shap_values, X_test, plot_type="dot", show=False)

# Customize the plot to match PLOS ONE style
plt.title("SHAP Summary Plot", fontsize=20, weight='bold', pad=20)
plt.xlabel('SHAP Value (impact on model output)', fontsize=16, weight='bold', labelpad=15)
plt.ylabel('Features', fontsize=16, weight='bold', labelpad=15)

# Customize ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Remove top and right spines for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)

# Adjust layout
plt.tight_layout()

# Save the figure with high resolution
plt.savefig("shap_values.png", dpi=300, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()
